IMPORTS AND CONFIGURATION

In [ ]:
import numpy as np
import pandas as pd
import os
import importlib
import sys
import copy
import diskcache

In [ ]:
backtesting_path = r'C:\Users\vinayak\Desktop\Backtesting'
print("path before", sys.path)
if backtesting_path not in sys.path:
    print("The backtesting folder was not in the systems path")
    sys.path.append(backtesting_path)
print("path after", sys.path)
print()

In [ ]:
# from scipy.optimize import minimize
from Modules import Plot
from Modules import Helpers
from Modules import Data as data
from Modules import Utility as util
from Modules import TradeAndLogics as TL
from Modules import Data_Processing as dp
import Dispersion.DispersionAdjustedFunctionality as daf
from Modules.enums import Option, LongShort, DB, FNO, Leg, OHLC, PTSL

In [ ]:
pwd()

In [ ]:
original_stdout = util.const(sys.stdout)
original_stdout.value

TRADING/ STRATEGY PARAMETERS

In [ ]:
start_date = '2023-07-01'
end_date = '2024-07-01'

In [ ]:
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [ ]:
index_symbol = 'BANKNIFTY'
expiry_type_near = 'I'
expiry_type_next = 'II'
risk_free_rate = 0.1 # (10% interest rate)
timeframe = 1 # mins
epsilon = 0.1
ICView = LongShort.Long

In [ ]:
index_symbol = 'BANKNIFTY'
index_lot_size = 15

In [ ]:
strategy_desc = f"IC_Analysis_BNF_PCA"
strategy_desc

Strategy Specific Data Dealing and Modifications

BACK TO NORMAL

In [ ]:
basket = daf.RawWeightedPortfolio()
basket.insert('HDFCBANK', 550, 27.04)
basket.insert('ICICIBANK', 700, 23.03)
basket.insert('KOTAKBANK', 400, 11.72)
basket.insert('SBIN', 750, 11.27)
basket.insert('AXISBANK', 625, 11.18)

# basket.insert('PNB', 8000, 0.91)
# basket.insert('AUBANK', 1000, 2.69)
# basket.insert('BANKBARODA', 2925, 1.84)
# basket.insert('FEDERALBNK', 5000, 1.68)
# basket.insert('IDFCFIRSTB', 7500, 1.08)
# basket.insert('INDUSINDBK', 500, 1.98)
# basket.insert('BANDHANBNK', 2800, 11.18)

In [ ]:
def get_indexes(start, end, next, next_fill):
    constituents_near = {}
    ohlc = OHLC.close
    index_near = daf.ticker(index_symbol, index_lot_size, True, start, end, expiry_type_near, True, timeframe, True, 0.1)
    index_near.initializeDispersion(constituents_near, False, 1)
    index_near.set_ohlc(ohlc)
    index_near.set_intention(ICView)
    for stock in basket.Symbols():
        constituents_near[stock] = daf.ticker(stock, basket.LotSize(stock), True, start, end, expiry_type_near, True, timeframe, False, 0.1)
        constituents_near[stock].initializeDispersion({}, True, basket.Weight(stock))
        constituents_near[stock].set_ohlc(ohlc)
        constituents_near[stock].set_intention(ICView.opposite())

    if next:
        constituents_next = {}
        index_next = daf.ticker(index_symbol, index_lot_size, True, start, end, expiry_type_next, next_fill, timeframe, True, 0.1)
        index_next.initializeDispersion(constituents_next, False, 1)
        index_next.set_ohlc(ohlc)
        index_next.set_intention(ICView)
        for stock in basket.Symbols():
            constituents_next[stock] = daf.ticker(stock, basket.LotSize(stock), True, start, end, expiry_type_next, next_fill, timeframe, False, 0.1)
            constituents_next[stock].initializeDispersion({}, True, basket.Weight(stock))
            constituents_next[stock].set_ohlc(ohlc)
            constituents_next[stock].set_intention(ICView.opposite())
        
        return index_near, index_next
    return index_near

In [ ]:
name = 'IC_NearIV'
file = open(f"LOGS_Generating_{name}.txt", 'w', buffering=1)
sys.stdout = file
header_flag = True
mode = 'w'
date_range = pd.date_range(start=start_date, end=end_date, freq='ME')
for month in date_range:
    start, end = month.replace(day=1).date(), month.date()
    index_near= get_indexes(start, end, False, False)
    portfolio = [index_near]
    portfolio += [c_near for c_near in index_near.components.values()]
    for timestamp in index_near.timestamps:
        index_near.get_ic_at(timestamp, True)  
        print("======================================================")
    ic = index_near.df_futures['ic'].to_frame()
    ic.to_csv(f'{name}.csv', mode=mode, header=header_flag)
    header_flag = False
    mode = 'a'

sys.stdout = original_stdout.value
file.close()
ic = pd.read_csv(f'{name}.csv', index_col=0, parse_dates=True)
Plot.save_plot(Plot.plot_df(ic, 'ic'), f'{name}.html')

In [ ]:
importlib.reload(dp)

In [ ]:
name = 'IC_NormallizedIV_Filled'
file = open(f"LOGS_Generating_{name}.txt", 'w', buffering=1)
sys.stdout = file
header_flag = True
mode = 'w'
date_range = pd.date_range(start=start_date, end=end_date, freq='ME')
for month in date_range:
    start, end = month.replace(day=1).date(), month.date()
    index_near, index_next = get_indexes(start, end, True, True)
    portfolio = [(index_near, index_next)]
    portfolio += [(c_near, c_next) for (c_near, c_next) in zip(index_near.components.values(), index_next.components.values())]
    for timestamp in index_near.timestamps:
        # if ((timestamp + pd.offsets.MonthEnd(0)) - timestamp).days < 7:
        #     index_near.df_futures.loc[timestamp, 'ic'] = 0
        #     continue
        for (ticker_near, ticker_next) in portfolio:
            # ticker_near.df_futures.loc[timestamp, 'iv'] = ticker_near, ticker_next)
            ticker_near.df_futures.loc[timestamp, 'iv'] = Helpers.get_normallized_iv(timestamp, ticker_near, ticker_next, True)
            print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")
        index_near.get_ic_at(timestamp, True)  
        print("======================================================")
    ic = index_near.df_futures['ic'].to_frame()
    ic.to_csv(f'{name}.csv', mode=mode, header=header_flag)
    header_flag = False
    mode = 'a'

sys.stdout = original_stdout.value
file.close()
ic = pd.read_csv(f'{name}.csv', index_col=0, parse_dates=True)
Plot.save_plot(Plot.plot_df(ic, 'ic'), f'{name}.html')

In [ ]:
name = 'IC_NormallizedIV_UnFilled'
file = open(f"LOGS_Generating_{name}.txt", 'w', buffering=1)
sys.stdout = file
header_flag = True
mode = 'w'
date_range = pd.date_range(start=start_date, end=end_date, freq='ME')
for month in date_range:
    start, end = month.replace(day=1).date(), month.date()
    index_near, index_next = get_indexes(start, end, True, False)
    portfolio = [(index_near, index_next)]
    portfolio += [(c_near, c_next) for (c_near, c_next) in zip(index_near.components.values(), index_next.components.values())]
    for timestamp in index_near.timestamps:
        # if ((timestamp + pd.offsets.MonthEnd(0)) - timestamp).days < 7:
        #     index_near.df_futures.loc[timestamp, 'ic'] = 0
        #     continue
        for (ticker_near, ticker_next) in portfolio:
            ticker_near.df_futures.loc[timestamp, 'iv'] = Helpers.get_normallized_iv(timestamp, ticker_near, ticker_next, True)
            print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")
        index_near.get_ic_at(timestamp, True)  
        print("======================================================")
    ic = index_near.df_futures['ic'].to_frame()
    ic.to_csv(f'{name}.csv', mode=mode, header=header_flag)
    header_flag = False
    mode = 'a'

sys.stdout = original_stdout.value
file.close()
ic = pd.read_csv(f'{name}.csv', index_col=0, parse_dates=True)
Plot.save_plot(Plot.plot_df(ic, 'ic'), f'{name}.html')

In [ ]:
name = 'IVWeighted_NearIV'
file = open(f"LOGS_Generating_{name}.txt", 'w', buffering=1)
sys.stdout = file
header_flag = True
mode = 'w'
date_range = pd.date_range(start=start_date, end=end_date, freq='ME')
for month in date_range:
    start, end = month.replace(day=1).date(), month.date()
    index_near= get_indexes(start, end, False, False)
    for timestamp in index_near.timestamps:
        numerator = index_near.get_iv_at(timestamp, True) 
        denominator = 0
        for component in index_near.components.values():
            denominator += component.get_iv_at(timestamp, True) * component.weight
        index_near.df_futures.loc[timestamp, 'weightediv'] = numerator/denominator
        print("======================================================")
    weightediv = index_near.df_futures['weightediv'].to_frame()
    weightediv.to_csv(f'{name}.csv', mode=mode, header=header_flag)
    header_flag = False
    mode = 'a'

sys.stdout = original_stdout.value
file.close()
weightediv = pd.read_csv(f'{name}.csv', index_col=0, parse_dates=True)
Plot.save_plot(Plot.plot_df(weightediv, 'weightediv'), f'{name}.html')

In [ ]:
name = 'IVWeighted_NormallizedIV_Filled'
file = open(f"LOGS_Generating_{name}.txt", 'w', buffering=1)
sys.stdout = file
header_flag = True
mode = 'w'
date_range = pd.date_range(start=start_date, end=end_date, freq='ME')
for month in date_range:
    start, end = month.replace(day=1).date(), month.date()
    index_near, index_next = get_indexes(start, end, True, True)
    for timestamp in index_near.timestamps:
        numerator = Helpers.get_normallized_iv(timestamp, index_near, index_next, True)
        denominator = 0
        print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")  
        for component_near, component_next in zip(index_near.components.values(), index_next.components.values()):
            denominator += Helpers.get_normallized_iv(timestamp, component_near, component_next, True) * component_near.weight
            print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")  
        index_near.df_futures.loc[timestamp, 'weightediv'] = numerator/denominator
        print("======================================================")
    weightediv = index_near.df_futures['weightediv'].to_frame()
    weightediv.to_csv(f'{name}.csv', mode=mode, header=header_flag)
    header_flag = False
    mode = 'a'

sys.stdout = original_stdout.value
file.close()
weightediv = pd.read_csv(f'{name}.csv', index_col=0, parse_dates=True)
Plot.save_plot(Plot.plot_df(weightediv, 'weightediv'), f'{name}.html')

In [ ]:
name = 'IVWeighted_NormallizedIV_UnFilled'
file = open(f"LOGS_Generating_{name}.txt", 'w', buffering=1)
sys.stdout = file
header_flag = True
mode = 'w'
date_range = pd.date_range(start=start_date, end=end_date, freq='ME')
for month in date_range:
    start, end = month.replace(day=1).date(), month.date()
    index_near, index_next = get_indexes(start, end, True, False)
    for timestamp in index_near.timestamps:
        numerator = Helpers.get_normallized_iv(timestamp, index_near, index_next, True)
        denominator = 0
        print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")  
        for component_near, component_next in zip(index_near.components.values(), index_next.components.values()):
            denominator += Helpers.get_normallized_iv(timestamp, component_near, component_next, True) * component_near.weight
            print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")  
        index_near.df_futures.loc[timestamp, 'weightediv'] = numerator/denominator
        print("======================================================")
    weightediv = index_near.df_futures['weightediv'].to_frame()
    weightediv.to_csv(f'{name}.csv', mode=mode, header=header_flag)
    header_flag = False
    mode = 'a'

sys.stdout = original_stdout.value
file.close()
weightediv = pd.read_csv(f'{name}.csv', index_col=0, parse_dates=True)
Plot.save_plot(Plot.plot_df(weightediv, 'weightediv'), f'{name}.html')

In [ ]:
# do one without filling too

In [14]:
def calculate_rsi(data, window=15):
    """
    Calculate the Relative Strength Index (RSI) for a given dataset.
    
    Parameters:
    - data: pandas Series or DataFrame column containing price data.
    - window: Look-back period for RSI calculation (default is 14).
    
    Returns:
    - pandas Series containing RSI values.
    """
    # Calculate price changes
    delta = data.diff()
    
    # Separate gains and losses
    gain = (delta.where(delta > 0, 0))
    loss = (-delta.where(delta < 0, 0))
    
    # Calculate the average gain and loss
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    # Calculate Relative Strength (RS)
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 1 - (1 / (1 + rs))
    
    return rsi

In [ ]:
rsi = calculate_rsi(ic, 15)

In [ ]:
rsi

In [ ]:
rsi = rsi/100
ic['rsi'] = rsi
ic['upper'] = rsi[rsi > 0.7]
ic['lower'] = rsi[rsi < 0.3]

In [ ]:
figg = Plot.plot_df(ic, *ic.columns)
Plot.save_plot(figg, 'ic_rsi.html')

In [ ]:
ic_15, _ = dp.get_continuous_excluding_market_holidays(ic, False, 15)

In [ ]:
ic_15 = ic_15['ic'].to_frame()

In [ ]:
rsi_2 = calculate_rsi(ic_15, 2)/100
ic_15['rsi_2'] = rsi_2
ic_15['upper'] = rsi_2[rsi_2 > 0.7]
ic_15['lower'] = rsi_2[rsi_2 < 0.3]
Plot.save_plot(Plot.plot_df(ic_15, *ic_15.columns), 'ic_15_rsi_2.html')

In [ ]:
ic = pd.read_csv(r, index_col = 0, parse_dates=True)

In [ ]:
from Modules import Data_Processing as dp

In [ ]:
ic_15 = dp.get_resampled_data(r'c:\Users\vinayak\Desktop\Backtesting\Dispersion\Optimizations\ICs\IC_NormallizedIV_Filled.csv', 15)

In [ ]:
ic_15['ic_smoothened'] = ic_15.ewm(span=3).mean()

In [ ]:
ic_15['rsi_2'] = calculate_rsi(ic_15['ic_smoothened'], 2)

In [ ]:
ic_15

In [ ]:
ic_15['rsi_upper'] = ic_15['rsi_2'][ic_15['rsi_2'] > 0.9]
ic_15['rsi_lower'] = ic_15['rsi_2'][ic_15['rsi_2'] < 0.1]

In [ ]:
fig = Plot.plot_df(ic_15, *(ic_15.columns))

In [ ]:
Plot.save_plot(fig, 'ic_15_rsi_2_smoothened.html')